<a href="https://colab.research.google.com/github/akshayanant/googlecolab/blob/master/madlibs_data_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
from pycocotools.coco import COCO


In [ ]:
person_activities = json.load(open('./tr_person_activities.json', 'r'))['tr_person_activities']
person_attributes = json.load(open('./tr_person_attributes.json', 'r'))['tr_person_attributes']
person_locations = json.load(open('./tr_person_locations.json', 'r'))['tr_person_locations']


In [ ]:
coco = COCO('./instances_val2014.json')


loading annotations into memory...
Done (t=5.71s)
creating index...
index created!


In [ ]:

def generate_data(data_list):
  images = []
  boxes = []
  for data in data_list:
    person_id_list = data['person_id_list']
    if(len(person_id_list)>1):
      continue
    image_id = data['image_id']
    prompt = data['prompt']
    fitbs = data['fitbs']
    helping_verb = prompt.split()[-1]
    start = ' '.join(word for word in prompt.split()[:-1])
    link = ' who ' + helping_verb 
    url = coco.loadImgs(ids=[image_id])[0]['coco_url']
    bbox_id = person_id_list[0]
    ann = coco.loadAnns(bbox_id)[0]
    
    flag = False
    for fitb in fitbs:
      if len(fitb.split())>5:
        flag=True
        ref_exp = '{0} {1}.'.format(start + link, fitb)
        img_obj = {
              'fileName' : url,
              'reference_pairs': [
                                    {
                                    'boundingBox_id' : bbox_id,
                                    'reference' : ref_exp
                                    }
                                ]
            }
        images.append(img_obj)
    if flag:
      bbox = ann['bbox']
      box_obj = {
        'box_id' : bbox_id,
        'x' : bbox[0],
        'y' : bbox[1],
        'width' : bbox[2],
        'height' : bbox[3]
      }
    boxes.append(box_obj)
  return {
      'images' : images,
      'boxes' : boxes
  }
          





In [ ]:
data_list = []
data_list.extend(person_activities)
data_list.extend(person_attributes)
data_list.extend(person_locations)
data = generate_data(data_list)


In [ ]:
with open('./Results.json','w') as file:
  json.dump(data,file)